### Задание 1

Для датафрейма log из материалов занятия создать столбец source_type по следующим правилам:

- если источник traffic_source равен yandex или google, то в source_type ставится organic
- для источников paid и email из России - ставим ad
- для источников paid и email не из России - ставим other
- все остальные варианты берем из traffic_source без изменений

In [31]:
import pandas as pd

In [32]:
def traffic_type(df):
    
    # Параметр traffic_source равен yandex или google
    if df['traffic_source'] == 'yandex' or df['traffic_source'] == 'google':
        return 'organic'
    # Параметр traffic_source равен paid или email
    elif df['traffic_source'] == 'paid' or df['traffic_source'] == 'email':
        if df['region'] == 'Russia':
            return 'ad'
        elif df['region'] != 'Russia':
            return 'other'
    # Все остальные значения параметра traffic_source
    else:
        return df['traffic_source']

In [33]:
visit_log = pd.read_csv('visit_log.csv', sep = ';')

visit_log['traffic_type'] = visit_log.apply(traffic_type, axis = 1)

visit_log.sample(10)

,timestamp,visit_id,url,region,user_id,traffic_source,traffic_type
14965,1550070143,42fa26ef5c,https://host.ru/8fe7c39e5c598bbc54,Ukraine,d86143e506,direct,direct
14338,1550066326,2765ba4a37,https://host.ru/23c64743ad842b9311,Russia,1c6f980a49,paid,ad
18215,1550090002,457406ebb4,https://host.ru/186c4d6b47dbde0e74,Russia,6ef07e14fd,email,ad
4341,1550006784,f972932e94,https://host.ru/ac2d98a5a63e7c6e28,Russia,2d1a750fc7,direct,direct
16686,1550080452,5bafdc826e,https://host.ru/2fcb223266880ccf57,Russia,d2afe584df,yandex,organic
997,1549986627,6418268f98,https://host.ru/ae8a036d7f44d0bcd9,Australia,06af703c27,yandex,organic
1419,1549989089,7a5a2b7d1c,https://host.ru/f2297425e1328e9e35,Russia,28d60ec3e0,direct,direct
10824,1550045305,9314ade80b,https://host.ru/f2297425e1328e9e35,Russia,529180bd75,direct,direct
5652,1550014689,f1e7342687,https://host.ru/53b1649b8344e663e9,Belarus,cde5efa21e,yandex,organic
16787,1550081138,033ae627b2,https://host.ru/ef57543002ce2a0f2a,Russia,1d1db3bed2,paid,ad


### Задание 2

В файле URLs.txt содержатся url страниц новостного сайта. Необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполнить следующие действия:

- Прочитать содержимое файла с датафрейм
- Отфильтровать страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [34]:
import re

In [35]:
URLs = pd.read_csv('URLs.txt')
url_news = URLs[URLs.url.str.contains(r'\/\d{8}\-')]
url_news.head()

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


### Задание 3

Используя файл с оценками фильмов ml-latest-small/ratings.csv, посчитать среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [40]:
movies = pd.read_csv('ml-latest-small/ratings.csv')
# movies.head()

# Выделение пользователей, которые поставили более 100 оценок
score_count= movies.groupby('userId').count()
more_100_scores = score_count[score_count.movieId > 100]

# Удаление столбцов с "одинаковыми" значениями количества оценок
userId = more_100_scores.drop(['movieId', 'rating', 'timestamp'], axis = 1)

# Добавленеи столбца timestamp
full_table = userId.merge(movies, on = 'userId', how = 'left')

# Определение минимального и максимального значения timestamp
lifetime = full_table.groupby('userId').agg({'timestamp':['min', 'max']}).reset_index()
# # b = count100.groupby('userId').agg(['min', 'max'])['timestamp']

# Расчет среднего срока жизни каждого пользователя
lifetime['lifetime'] = lifetime['timestamp']['max'] - lifetime['timestamp']['min']
lifetime.lifetime.mean()

40080507.4496124

### Задание 4

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

In [41]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [42]:
# Создание таблицы с тремя типами выручки для каждого client_id без указания адреса клиента. Вартант 1

tables = [rzd, auto, air]
pd.concat([df.set_index('client_id') for df in tables], axis = 1)

,rzd_revenue,auto_revenue,air_revenue
client_id,,,
111,1093.0,NaN,NaN
112,2810.0,NaN,NaN
113,10283.0,57483.0,NaN
114,5774.0,83.0,NaN
115,981.0,912.0,81.0
116,NaN,4834.0,4.0
117,NaN,98.0,13.0
118,NaN,NaN,173.0


In [43]:
# Создание таблицы с тремя типами выручки для каждого client_id без указания адреса клиента. Вартант 2

sum_table = pd.concat([rzd, auto, air])
sum_table.pivot_table( index = 'client_id', values = ['rzd_revenue', 'auto_revenue', 'air_revenue'])

,air_revenue,auto_revenue,rzd_revenue
client_id,,,
111,NaN,NaN,1093.0
112,NaN,NaN,2810.0
113,NaN,57483.0,10283.0
114,NaN,83.0,5774.0
115,81.0,912.0,981.0
116,4.0,4834.0,NaN
117,13.0,98.0,NaN
118,173.0,NaN,NaN


In [44]:
# Создание таблицы по типам выручки с указанием адреса клиента

a = pd.merge(client_base, rzd, on = 'client_id', how = 'outer')
b = pd.merge(a, auto, on = 'client_id', how = 'outer')
pivot_tabel = pd.merge(b, air, on = 'client_id', how = 'outer')
pivot_tabel

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0
